**The below question were asked during the interview**

### Python Problem2

In [1]:

# Problem-1
data_dict = [
{ "x": "y"},
{ "1": "2"},
{ "t": "y"}
]

# Output: {"x": "y", "1": "2", "t": "y"}

def reformat(data_dict):
    response = {}
    for row in data_dict:
        for key,val in row.items():
            response[key] = val
    return response
print(reformat(data_dict))

{'x': 'y', '1': '2', 't': 'y'}


### Python Problem2

In [3]:
x = [ 'a', 'b', 'c' ] 
y = [ 'a', 'b' ]
# how to get x - y operation
res1 = [ele for ele in x if ele not in y]
print(res1)
res2 = set(x) - set(y)
print(res2)

['c']
{'c'}


### Pyspark Problem

In [82]:
#Pyspark:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import Window as WN

In [6]:
dataset = [
    {"id":"abc","amount":10},
    {"id":"def","amount":20}
]
spark = SparkSession.builder.master("local[2]").appName("US_Client").getOrCreate()

## Problem-1
```
+------+---+
|amount| id|
+------+---+
|    10|abc|
|    20|def|
+------+---+
```
**Output**
```
['{"amount":10,"id":"abc"}', '{"amount":20,"id":"def"}']
```

In [11]:
df = spark.createDataFrame(dataset)
df.show()

+------+---+
|amount| id|
+------+---+
|    10|abc|
|    20|def|
+------+---+



**output**

In [9]:
df.toJSON().collect()

['{"amount":10,"id":"abc"}', '{"amount":20,"id":"def"}']

### Problem2
Convert dataframe to below format
~~~
Output JSON 2:
{
"ID" : [ "abc", "def"],
"amount" : [ 10, 20]
}

~~~

In [13]:
# Convert DataFrame to dictionary
dict_data = df.groupBy().agg(FS.collect_list("id").alias("id"), FS.collect_list("amount").alias("amount")).collect()[0].asDict()


In [18]:
data_dict = df.groupBy().agg(FS.collect_list("id").alias("id"),FS.collect_list("amount").alias("amount"))

In [21]:
data_dict.collect()[0].asDict()

{'id': ['abc', 'def'], 'amount': [10, 20]}

In [26]:
newJson = '{"Name":"something","Url":"https://stackoverflow.com","Author":"jangcy","BlogEntries":100,"Caller":"jangcy"}'
df = spark.read.json(spark.sparkContext.parallelize([newJson]))
df.show(truncate=False)

+------+-----------+------+---------+-------------------------+
|Author|BlogEntries|Caller|Name     |Url                      |
+------+-----------+------+---------+-------------------------+
|jangcy|100        |jangcy|something|https://stackoverflow.com|
+------+-----------+------+---------+-------------------------+



### Problem2
Convert dataframe to below format
~~~
A table with following columns
Month			Sales
Apr-2020		10,000
May-2020		12,000
Jun-2020		11,400
~~~
**Output**:
~~~
Month			Sales		Growth%
Apr-2020		10,000		Null
May-2020		12,000		20
Jun-2020		11,400		-5
~~~

In [64]:
dataset = [
    {"month":"Apr-2020","sales":10000},
     {"month":"May-2020","sales":12000},
     {"month":"Jun-2020","sales":11400},
]
sale_df = spark.createDataFrame(dataset)
sale_df.show()


+--------+-----+
|   month|sales|
+--------+-----+
|Apr-2020|10000|
|May-2020|12000|
|Jun-2020|11400|
+--------+-----+



In [65]:

from pyspark.sql.types import DateType
def convert_date(col):
    from datetime import datetime
    part = col.split("-")
    date_str = part[-1] + "-" +  part[0] + "-01"
    date = datetime.strptime(date_str, '%Y-%b-%d')
    return date

# Converting function to UDF 
convertUDF = FS.udf(lambda z: convert_date(z),DateType())
# sale_df.withColumn("month1",FS.to_date('month1','m-Y')).show()

In [70]:
sales_sf2 = sale_df.select("sales",convertUDF(FS.col('month')).alias("month"))

In [28]:
from pyspark.sql import Window as wn

In [71]:
sales_sf2.createOrReplaceTempView("sales")

In [72]:
spark.sql("select * from sales").show()

+-----+----------+
|sales|     month|
+-----+----------+
|10000|2020-04-01|
|12000|2020-05-01|
|11400|2020-06-01|
+-----+----------+



In [77]:
sql = """
with SalesData as (
select month,sales,
lag(sales) over(order by month) as previous_sale from sales
)
select month,sales,
CASE
WHEN previous_sale IS NULL THEN NULL
ELSE ROUND(((sales - previous_sale)/previous_sale * 100),2)
END AS growth
from SalesData;
"""



spark.sql(sql).show()

+----------+-----+------+
|     month|sales|growth|
+----------+-----+------+
|2020-04-01|10000|  NULL|
|2020-05-01|12000|  20.0|
|2020-06-01|11400|  -5.0|
+----------+-----+------+



In [99]:
## Similarly with Pyspakr Dataframe
sales_sf2.withColumn("previous_sale",FS.lag('sales').over(WN.partitionBy().orderBy(FS.col('month').asc()))).\
select("month","sales",FS.col('previous_sale'),FS.when(FS.col('previous_sale') == 'null','null').\
       otherwise((FS.col('sales')- FS.col('previous_sale'))/FS.col('previous_sale') * 100).alias("growth")).show()


+----------+-----+-------------+------+
|     month|sales|previous_sale|growth|
+----------+-----+-------------+------+
|2020-04-01|10000|         NULL|  NULL|
|2020-05-01|12000|        10000|  20.0|
|2020-06-01|11400|        12000|  -5.0|
+----------+-----+-------------+------+



In [100]:
data = [(1, [1,2,3,4,5,6]), (2, [7,8,9,10,11,12,13])]
df_arr = spark.createDataFrame(data, ["dno", "eno"])
df_arr.show()

+---+--------------------+
|dno|                 eno|
+---+--------------------+
|  1|  [1, 2, 3, 4, 5, 6]|
|  2|[7, 8, 9, 10, 11,...|
+---+--------------------+



In [105]:
df_exploded = df_arr.withColumn("eno", FS.explode(FS.col("eno")))
df_exploded.show()

+---+---+
|dno|eno|
+---+---+
|  1|  1|
|  1|  2|
|  1|  3|
|  1|  4|
|  1|  5|
|  1|  6|
|  2|  7|
|  2|  8|
|  2|  9|
|  2| 10|
|  2| 11|
|  2| 12|
|  2| 13|
+---+---+



In [106]:
df_exploded = df_exploded.withColumn("group", (col("eno") - 1) % 4)
df_exploded.show()

+---+---+-----+
|dno|eno|group|
+---+---+-----+
|  1|  1|    0|
|  1|  2|    1|
|  1|  3|    2|
|  1|  4|    3|
|  1|  5|    0|
|  1|  6|    1|
|  2|  7|    2|
|  2|  8|    3|
|  2|  9|    0|
|  2| 10|    1|
|  2| 11|    2|
|  2| 12|    3|
|  2| 13|    0|
+---+---+-----+



In [108]:
df_transformed = df_exploded.groupby("dno", "group").agg(FS.collect_list("eno").alias("eno"))

# Drop the 'group' column
df_transformed = df_transformed.drop("group")

# Show the transformed dataframe
df_transformed.show(truncate=False)

+---+-------+
|dno|eno    |
+---+-------+
|1  |[4]    |
|1  |[1, 5] |
|1  |[2, 6] |
|1  |[3]    |
|2  |[8, 12]|
|2  |[7, 11]|
|2  |[9, 13]|
|2  |[10]   |
+---+-------+

